In [23]:
import numpy as np
from tqdm import tqdm

In [24]:
lamb1 = 0.01
lamb2 = 0.1
#Q depends on the data
#phi_i too

In [25]:
def adapted_norm(g, X):
    return np.linalg.norm(g) + np.linalg.norm(X, ord ='fro')

In [26]:
def phi(X, phi_i):
    n = len(phi_i)
    res = np.zeros((n,1))
    for i in range(n):
        res[i] = np.trace(np.dot(X.T, np.dot(phi_i[i],phi_i[i].T)))
    return res

def phi_star(gamma, phi_i):
    n = len(phi_i)
    res = np.zeros((n,n))
    for i in range(n):
        res += gamma[i]*np.dot(phi_i[i],phi_i[i].T)
    return res

In [27]:
def proj_Sn2(A):
    n = A.shape[0]
    vals, vecs = np.linalg.eig(A)
    idx = np.argsort(-vals) 
    sorted_vals = vals[idx]
    sorted_vecs = vecs[:, idx]
    P = sorted_vecs
    #D = np.diag(sorted_vals)
    rg = 0
    while sorted_vals[rg] >= 0 and rg < len(sorted_vals):
        rg +=1
    if rg > 0:
        P_alpha = P[:,:rg].T
    else:
        P_alpha = np.array([[0]])
    D_alpha = np.diag(sorted_vals[:rg])
    alpha = idx[:rg]
    eta = np.zeros((rg, n - rg))
    for i in range(rg):
        for j in range(rg, n-rg):
            ind1, ind2 = idx[i], idx[j]
            eta[ind1, ind2] = vals[ind1]/(vals[ind1] - vals[ind2])
    omega = np.block([[np.ones((rg, rg)), eta], [eta.T, np.zeros((n-rg, n-rg))]])
    res = np.dot(P_alpha, np.dot(D_alpha, P_alpha.T))
    return (res, P_alpha, omega, alpha, eta) 

import numpy as np

def proj_Sn(A):
    n = A.shape[0]
    # Calcul des valeurs et vecteurs propres
    vals, vecs = np.linalg.eigh(A)  # Utilisez eigh pour les matrices symétriques
    idx = np.argsort(-vals)         # Trier par valeurs propres décroissantes
    sorted_vals = vals[idx]
    sorted_vecs = vecs[:, idx]
    #print(vals)
    # Déterminer le rang de la partie positive
    rg = np.sum(sorted_vals >= 0)

    # Construction de P_alpha
    if rg > 0:
        P_alpha = sorted_vecs[:, :rg]
    else:
        P_alpha = np.zeros((n, 0))  
    #P_k = vecs 
    # Diagonale D_alpha pour les valeurs propres positives
    D_alpha = np.diag(sorted_vals[:rg])

    # Calcul de la matrice omega
    if rg < n:
        eta = np.zeros((rg, n - rg))
        positive_vals = sorted_vals[:rg]
        non_positive_vals = sorted_vals[rg:]
        for i in range(rg):
            for j in range(n - rg):
                eta[i, j] = positive_vals[i] / (positive_vals[i] - non_positive_vals[j])
        omega = np.block([[np.eye(rg), eta], [eta.T, np.zeros((n-rg, n-rg))]])
    else:
        omega = np.eye(n)  # Cas où toutes les valeurs sont positives

    # Calcul de la projection résultante
    #print(P_alpha.shape, D_alpha.shape)
    res = P_alpha @ D_alpha @ P_alpha.T  # Utiliser l'opérateur @ pour les produits matriciels
    #print(res.shape)
    return res, vecs, omega, idx[:rg], eta


In [28]:
def R(gamma, X, Q, phi_i, z):
    n = len(phi_i)
    res1 = (1/(2*lamb2))*np.dot(Q, gamma) - (1/(2*lamb2))*z - phi(X, phi_i)
    res2 = X - proj_Sn(X - (phi_star(gamma, phi_i) + lamb1*np.eye(n)))[0]
    #print(res1.shape, res2.shape)
    return (res1, res2)

In [29]:
def Pk_psik(gamma_k, X_k, phi_i, mu_k):
    n = len(phi_i)
    Z_k = X_k - (phi_star(gamma_k, phi_i) + lamb1*np.eye(n))
    res = proj_Sn(Z_k)
    res1 = res[1] #P_k
    #print(res1.shape)
    #res1 = res1.reshape(res1.shape[0],1)
    #print(phi_star(gamma_k, phi_i))
    #print(res1)
    #res12 = np.diag(res[2].reshape(1,-1)[0])
    #taille = res12.shape[0]
    #print(f"taille={taille}")
    #L_k = np.eye(taille)
    #for i in range(taille):
        #L_k[i,i] = res12[i,i]/(mu_k + 1 - res12[i,i])
    #P_tilde = np.kron(res1, res1)
    #print(res1.shape, P_tilde.shape, L_k.shape, res1, X_k, Z_k)
    #if res1.size == 0:
        #res2 = L_k
    #else:
        #res2 = np.dot(P_tilde.T, np.dot(L_k, P_tilde)) #T_k
        #print(res2.shape)
    alpha = res[3]
    eta = res[4]
    rg = len(alpha)
    xi = np.zeros((rg, n - rg))
    for i in range(rg):
        for j in range(rg, n-rg):
            xi[i,j] = eta[i,j]/(mu_k + 1 - eta[i,j])
    E = (1/mu_k)*np.ones((rg, rg))
    psi_k = np.block([[E, xi], [xi.T, np.zeros((n-rg, n-rg))]])
    return(res1, psi_k)

def op_Tk(S, Pk, psik):
    res = psik * np.dot(Pk.T, np.dot(S, Pk)) #(n,n) * (alpha,n),(n,n),(n,alpha) 
    res = np.dot(Pk.T, np.dot(res, Pk)) 
    return res

In [30]:
def a(mu_k, Pk, psik, rk1, rk2, phi_i):
    #rk1, rk2 = R(gamma_k, X_k, Q, phi_i, z)
    #mu_k = theta_k*adapted_norm(rk1, rk2)
    #T_k = Pk_Tk(gamma_k, X_k, phi_i, mu_k)[1]
    #print(rk1.shape, rk2.shape)
    res1 = -rk1 - (1/(mu_k + 1))*phi(rk2 + op_Tk(rk2, Pk, psik), phi_i)
    res2 = -rk2
    return (res1, res2)

def a_tilde(gamma_k, a_, Q, mu_k, P_k, psi_k, phi_i):
    a1, a2 = a_[0], a_[1]
    #rk1, rk2 = R(gamma_k, X_k, Q, phi_i, z)
    #mu_k = theta_k*adapted_norm(rk1, rk2)
    #T_k = Pk_Tk(gamma_k, X_k, phi_i, mu_k)[1]
    n = Q.shape[0]
    #print(Q.shape, T_k.shape )
    #res1 = np.linalg.inv((1/(2*lamb2))*Q + mu_k*np.eye(n) + phi(T_k.dot(phi_star(gamma_k, phi_i)), phi_i))
    #print(op_Tk(phi_star(gamma_k, phi_i), P_k, psi_k).shape, phi_star(gamma_k, phi_i).shape, P_k.shape, psi_k.shape)
    res1 = np.linalg.inv((1/(2*lamb2))*Q + mu_k*np.eye(n) + phi(op_Tk(phi_star(gamma_k, phi_i), P_k, psi_k), phi_i))
    res1 = np.dot(res1, a1)
    #P_k = Pk_Tk(gamma_k, X_k, phi_i, mu_k)[0]
    #psi_k = Pk_Tk(gamma_k, X_k, phi_i, mu_k)[2]
    res2 = 1/(mu_k + 1)*(a2 + op_Tk(a2, P_k, psi_k))
    return (res1, res2)

In [31]:
def grad_w(a_t, P_k, psi_k, phi_i):
    a_t1, a_t2 = a_t
    res1 = a_t1
    res2 = a_t2 - op_Tk(phi_star(a_t1, phi_i), P_k, psi_k)
    return(res1, res2)

In [32]:
def next_theta(theta_, _theta, beta0, beta1, beta2, alpha_1, alpha_2, thetak, delta_gammak, delta_Xk, rk1, rk2):
    #next_gamma = gamma_k + delta_gammak
    #next_X = X_k + delta_Xk
    #rk1, rk2 = R(next_gamma, next_X, Q, phi_i, z)
    pho_k = - np.dot(rk1.T, delta_gammak) - np.trace(np.dot(rk2.T, delta_Xk))
    norm_grad = adapted_norm(delta_gammak, delta_Xk)**2
    if pho_k >= alpha_2*norm_grad:
        return max(theta_, beta0*thetak)
    elif pho_k >= alpha_1*norm_grad:
        return beta1*thetak
    else:
        return min(_theta, beta2*thetak)

In [3]:
def OT(q2, gamma, wx, wy):
    res = (1/(2*lamb2))*(- np.sum(gamma.T*(wx+wy)) + q2)
    return res

In [1]:
def algo2(Q, phi_i, z, gamma_0, X_0, theta_0, q2, wx, wy, nb_iter = 1000, beta0 = 0.5, beta1 = 1.2, beta2 = 5, alpha_1 = 1e-6, alpha_2 = 1, theta_ = 5, _theta = 15):
    gamma_k = gamma_0
    X_k = X_0
    theta_k = theta_0
    rk1, rk2 = R(gamma_k, X_k, Q, phi_i, z)
    n = [np.inf]
    ind = []
    gammas = []
    ot = []
    for k in tqdm(range(nb_iter)):
    #for _ in range(nb_iter):
        #rk1, rk2 = R(gamma_k, X_k, Q, phi_i, z)
        mu_k = theta_k*adapted_norm(rk1, rk2)
        #T_k = Pk_Tk(gamma_k, X_k, phi_i, mu_k)[1]
        P_k, psi_k = Pk_psik(gamma_k, X_k, phi_i, mu_k)
        #psi_k = Pk_psik(gamma_k, X_k, phi_i, mu_k)[1]
        a_ =  a(mu_k, P_k, psi_k, rk1, rk2, phi_i)
        a_t =  a_tilde(gamma_k, a_, Q, mu_k, P_k, psi_k, phi_i)
        delta_gammak, delta_Xk = grad_w(a_t, P_k, psi_k, phi_i)
        #next_t = next_theta(theta_, _theta, beta0, beta1, beta2, alpha_1, alpha_2, theta_k, delta_gammak, delta_Xk, rk1, rk2)
        X_k += delta_Xk
        gamma_k += delta_gammak
        rk1, rk2 = R(gamma_k, X_k, Q, phi_i, z)
        next_t = next_theta(theta_, _theta, beta0, beta1, beta2, alpha_1, alpha_2, theta_k, delta_gammak, delta_Xk, rk1, rk2)
        theta_k = next_t
        #print(adapted_norm(rk1, rk2))
    #rk1, rk2 = R(gamma_k, X_k, Q, phi_i, z)
        #if adapted_norm(rk1, rk2) < 0.005:
            #break
        norm = adapted_norm(rk1, rk2)
        if norm < n[-1]:
            n.append(norm)
            ind.append(k)
            gammas.append(np.linalg.norm(gamma_k))
            ot.append(OT(q2, gamma_k, wx, wy))
    #norm_fin = adapted_norm(rk1, rk2)
    return(gamma_k, X_k, n, ind, gammas, ot)

In [2]:
import numpy as np

def gradient(gamma, X, Q, z, phi_i): #n'est pas correct, we whould have use lemma 4.3. to get J_k...
    n = len(phi_i)
    Phi = phi(X, phi_i)
    H = Phi.T @ X @ Phi
    g_gamma = (Q @ gamma - z) / (2 * lamb2) - np.diag(np.diag(H))
    #print(Phi.shape, gamma.T.shape)
    #print(np.diag(gamma.T[0]).shape)
    g = np.diag(gamma.reshape(1,-1)[0])
    #print(g_gamma)
    g_X = Phi.T @ g @ Phi + lamb1 * np.eye(n)  

    return g_gamma, g_X



def one_step_extragradient(gamma, X, Q, z, alpha, phi_i):
    gamma_inter = gamma - alpha *gradient(gamma, X, Q, z, phi_i)[0]
    X_inter = X - alpha * gradient(gamma, X, Q, z, phi_i)[1]
    
    gamma_next = gamma - alpha * gradient(gamma_inter, X_inter, Q, z, phi_i)[0]
    X_next = X - alpha * gradient(gamma_inter, X_inter, Q, z, phi_i)[1]
    return gamma_next, X_next

In [ ]:
def algo22(Q, phi_i, z, gamma_0, X_0, theta_0, nb_iter = 1000, beta0 = 0.5, beta1 = 1.2, beta2 = 5, alpha_1 = 1e-6, alpha_2 = 1, theta_ = 5, _theta = 15):
    gamma_k = gamma_0
    X_k = X_0
    theta_k = theta_0
    v_gamma_k = gamma_0
    v_X_k = X_0
    rk1, rk2 = R(gamma_k, X_k, Q, phi_i, z)
    n = []
    ind = []
    for _ in tqdm(range(nb_iter)):
    #for _ in range(nb_iter):
        #rk1, rk2 = R(gamma_k, X_k, Q, phi_i, z)
        mu_k = theta_k*adapted_norm(rk1, rk2)
        #T_k = Pk_Tk(gamma_k, X_k, phi_i, mu_k)[1]
        P_k, psi_k = Pk_psik(gamma_k, X_k, phi_i, mu_k)
        #psi_k = Pk_psik(gamma_k, X_k, phi_i, mu_k)[1]
        a_ =  a(mu_k, P_k, psi_k, rk1, rk2, phi_i)
        a_t =  a_tilde(gamma_k, a_, Q, mu_k, P_k, psi_k, phi_i)
        delta_gammak, delta_Xk = grad_w(a_t, P_k, psi_k, phi_i)
        #next_t = next_theta(theta_, _theta, beta0, beta1, beta2, alpha_1, alpha_2, theta_k, delta_gammak, delta_Xk, rk1, rk2)

        candidate_X = X_k + delta_Xk
        candidate_gamma = gamma_k + delta_gammak
        rk11, rk21 = R(candidate_gamma, candidate_X, Q, phi_i, z)
        norm1 = adapted_norm(rk11, rk21)

        v_gamma_k, v_X_k = one_step_extragradient(v_gamma_k, v_X_k, Q, z, theta_k, phi_i)
        rk12, rk22 = R(v_gamma_k, v_X_k, Q, phi_i, z)
        norm2 = adapted_norm(rk12, rk22)
        
        if norm1 < norm2:
            gamma_k = candidate_gamma
            X_k = candidate_X
            n.append(norm1)
            rk1, rk2 = rk11, rk21
        else:
            gamma_k = v_gamma_k
            X_k = v_X_k
            rk1, rk2 = rk12, rk22
            n.append(norm2)
        next_t = next_theta(theta_, _theta, beta0, beta1, beta2, alpha_1, alpha_2, theta_k, delta_gammak, delta_Xk, rk1, rk2)
        theta_k = next_t
    #norm_fin = adapted_norm(rk1, rk2)
    return(gamma_k, X_k, n, ind)